In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from gensim import models
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
import os
from pypapago import Translator

In [2]:
eat_info = pd.read_csv("../data/건대_군자_광진_total_detail.csv")
eat_review = pd.read_csv("../data/건대_군자_광진_total_review_list.csv")

In [3]:
eat_info

,Unnamed: 0,가게이름,가게 주소,음식 종류,가격 범위,리뷰개수,평점
0,0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,만원 미만,72,4.5
1,1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,만원 미만,94,4.2
2,2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,만원 미만,40,4.0
3,3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,만원 미만,13,4.0
4,4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,만원 미만,9,4.0
...,...,...,...,...,...,...,...
905,905,마우로아,서울특별시 광진구 능동로 320 1F,칵테일 / 와인,3만원-4만원,10,4.2
906,906,글래시,서울특별시 광진구 능동로 23-14 1F,칵테일 / 와인,3만원-4만원,19,3.5
907,907,리바,서울특별시 광진구 워커힐로 177 비스타 워커힐 1F,칵테일 / 와인,4만원 이상,4,NaN
908,908,몽,서울특별시 광진구 아차산로33길 25,칵테일 / 와인,4만원 이상,1,NaN


In [4]:
eat_review

,Unnamed: 0,가게이름,리뷰아이디,리뷰,맛평가
0,0,원조할아버지손두부,달려요달려,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,맛있다
1,1,원조할아버지손두부,ultraus,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,맛있다
2,2,원조할아버지손두부,세상의 모든 음식을 정복하겠어,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,맛있다
3,3,원조할아버지손두부,조선의장희빈,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,맛있다
4,4,원조할아버지손두부,미콩,드디어 먹어본 아차산 순두부!! 같이 가 준 친구들이 이미 전에도 맛본 경험이 있어...,맛있다
...,...,...,...,...,...
5150,5150,리바,나뮤,소리 너무너무너무 시끄러워요. 클럽마냥 소리질러야 대화가 됩니다~ 야경은 예뻐요,별로
5151,5151,리바,Consumericus,비비드아워로 방문.종류는 아주 다양하다거나 신라 정도의 퀄리티는 아니지만 뷰가 예쁘...,괜찮다
5152,5152,리바,썸머쏭쏭,"• 딱! 괜찮은 정도에요• 물론, 뷰+분위기는 굉장히 힙하나,• 팥빙수는 정말.. ...",괜찮다
5153,5153,몽,Tiffajy,분위기 어둑지고 아늑한 곳. 인도풍? 칵테일바 인데 분수도 있고 신발벗고 편히 앉아...,맛있다


In [5]:
eat_info.columns

Index(['Unnamed: 0', '가게이름', '가게 주소', '음식 종류', '가격 범위', '리뷰개수', '평점'], dtype='object')

In [6]:
eat_info.rename(columns={'가게이름': 'name'}, inplace =True)
eat_info.rename(columns={'리뷰': 'review'}, inplace =True)
eat_info.rename(columns={'평점': 'rating'}, inplace =True)
eat_info.rename(columns={'리뷰개수': 'rvw_cnt'}, inplace =True)
eat_info.rename(columns={'가게 주소': 'address'}, inplace =True)
eat_info.rename(columns={'음식 종류': 'category'}, inplace =True)
eat_info.rename(columns={'가격 범위': 'price'}, inplace =True)




eat_review.rename(columns={'가게이름': 'res_name'}, inplace =True)
eat_review.rename(columns={'리뷰아이디':  'user_name'}, inplace =True)
eat_review.rename(columns={'리뷰':  'review'}, inplace =True)
eat_review.rename(columns={'맛평가':  'rating'}, inplace =True)

eat_info.drop(['Unnamed: 0'],axis=1, inplace=True)
eat_review.drop(['Unnamed: 0'],axis=1, inplace=True)



In [7]:
eat_info.columns

Index(['name', 'address', 'category', 'price', 'rvw_cnt', 'rating'], dtype='object')

In [8]:
eat_info = eat_info[eat_info['name'].notnull()]
eat_review = eat_review[eat_review['user_name'].notnull()]
eat_info.reset_index(drop=True,inplace=True)
eat_review.reset_index(drop=True,inplace=True)

In [9]:
#delete overlapped data
for id in eat_info.index:
    if list(eat_info['name']).count(eat_info.loc[id,'name'])>1:
        eat_info.drop(id,inplace=True)

In [10]:
#label encoding, Place_name -> p_id, User_name -> u_id
le = LabelEncoder()
le.fit(eat_info['name'])
eat_info['p_id'] = le.transform(eat_info['name'])
eat_review['p_id'] = le.transform(eat_review['res_name'])

user_le = LabelEncoder()
user_le.fit(eat_review['user_name'])
eat_review['u_id'] = user_le.transform(eat_review['user_name'])

/var/folders/39/19s9x2053p9bhn6svbpwcm_h0000gn/T/ipykernel_43148/1585170050.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eat_review['p_id'] = le.transform(eat_review['res_name'])
/var/folders/39/19s9x2053p9bhn6svbpwcm_h0000gn/T/ipykernel_43148/1585170050.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eat_review['u_id'] = user_le.transform(eat_review['user_name'])


In [11]:
# delete punctuation marks
def delete_punctuation_marks_review(review):
    review = review.replace('.', '. ');
    review = review.replace('!', '! ');
    review = review.replace('(', '( ');
    review = review.replace(')', ') ');
    review = review.replace('^', '');
    review = review.replace('*', '');
    review = review.replace('-', ' ');
    review = review.replace('\n', '. ');
    review = review.replace('\"', '. ');
    return review

In [12]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,p_id
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,만원 미만,72,4.5,553
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,만원 미만,94,4.2,383
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,만원 미만,40,4.0,78
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,만원 미만,13,4.0,715
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,만원 미만,9,4.0,45


In [13]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,원조할아버지손두부,달려요달려,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,맛있다,553,715
1,원조할아버지손두부,ultraus,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,맛있다,553,453
2,원조할아버지손두부,세상의 모든 음식을 정복하겠어,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,맛있다,553,1080
3,원조할아버지손두부,조선의장희빈,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,맛있다,553,1479
4,원조할아버지손두부,미콩,드디어 먹어본 아차산 순두부!! 같이 가 준 친구들이 이미 전에도 맛본 경험이 있어...,맛있다,553,954


In [14]:
eat_info['reviews'] = ''


In [15]:
# from requests import Request
# from requests import Session


# def KOtoEN(review):
#     s=Session()

#     headers ={
#         "X-Naver-Client-Id": "NsHid2ZsKcP5QclyL66L",
#         "X-Naver-Client-Secret": "OPb4L85YOo",
#     }

#     payload ={
#         'source':'ko',
#         'target':'en',
#         'text': review,
#     }

#     url ="https://openapi.naver.com/v1/papago/n2mt"
#     # "https://openapi.naver.com/v1/language/translate"

#     req =Request('POST', url, data=payload, headers=headers)
#     prepped =req.prepare()
#     res =s.send(prepped)
#     print(res.json())
#     # result =res.json()['message']['result']['translatedText']
#     return result

In [16]:
review_1=eat_review['review'][1]
review_1

'아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는데 회전이 워낙 빨라 십분 이상 대기한 적이 별로 없는 것 같다. 포장도 가능해서 줄이 길면 그냥 포장해도 된다(포장시 두부에서 물이 빠져 좀 단단해지는 감은 있다) 모두부, 순두부, 콩국수가 있다. 콩국수는 걸쭉한 콩국물이 찐이고 순두부와 모두부 모두 갓 만든 듯한 고소함과 탱글한 질감을 보여준다. 곁들여 나오는 젓갈이 또 별미다. 막걸리 한잔 안할 수가 없는 곳.'

In [17]:
# KOtoEN(review_1)

In [18]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = KOtoEN(review_pre)
#     except:
#         review_ts = ''

#     print(review_ts)
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [19]:
# ts = Translator()

# ts.translate(review_1, source='ko', target='en', verbose=False)

JSONDecodeError: [Errno Expecting value]  <!doctype html><html lang="ko" data-placeholders-focus="false"><head><!-- Google Tag Manager --><script>!function(e,t,a,n,g){e[n]=e[n]||[],e[n].push({"gtm.start":(new Date).getTime(),event:"gtm.js"});var m=t.getElementsByTagName(a)[0],r=t.createElement(a);r.async=!0,r.src="https://www.googletagmanager.com/gtm.js?id=GTM-P4MDNNV",m.parentNode.insertBefore(r,m)}(window,document,"script","dataLayer")</script><!-- End Google Tag Manager --><meta http-equiv="Content-Type" content="text/html" charset="utf-8"/><meta http-equiv="X-XSS-Protection" content="1"/><meta http-equiv="X-UA-Compatible" content="IE=edge"/><meta name="referrer" content="origin"/><meta name="format-detection" content="telephone=no"/><meta name="theme-color" content="#ffffff"/><!-- mobile compatible --><meta name="viewport" content="width=device-width,initial-scale=1,minimum-scale=1,user-scalable=no"/><!--<meta name="mobile-web-app-capable" content="yes" />
    <meta name="apple-mob

In [20]:
# ts = Translator()
# eat_review['review_eng'] = ''

# #translating review
# for id in tqdm(eat_review.index):
#     review = eat_review.loc[id,'review']
    
#     if type(review)==float:
#         continue
        
#     review_pre = delete_punctuation_marks_review(review)
    
#     try:
#         review_ts = ts.translate(review_pre, source='ko', target='en', verbose=False)
#     except:
#         review_ts = ''
    
#     eat_review.loc[id,'review_eng'] = review_ts

In [25]:
for id in eat_info['p_id']:
    review_all = ''
    
    for review in eat_review[eat_review['p_id']==id]['review']:
        review_all+=review +'. '
        
    eat_info.loc[eat_info['p_id']==id,'reviews'] = review_all

In [34]:
eat_info.head()


,name,address,category,price,rvw_cnt,rating,p_id,reviews
0,원조할아버지손두부,서울특별시 광진구 자양로 324,한정식 / 백반 / 정통 한식,5000,72,4.5,553,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...
1,서북면옥,서울특별시 광진구 자양로 199-1,국수 / 면 요리,5000,94,4.2,383,서울특별시 광진구 구의2동에 위치한 서북면옥에서 물냉면(9천원)과 접시만두 6조각(...
2,궤도에오르다,서울특별시 광진구 광나루로 416 건희빌딩 1F,기타 한식,5000,40,4.0,78,기본에 충실해서 더 맛있었던 핫도그였습니다. 들어가서 음식이 나오기까지는 꽤 시간이...
3,콩이네,서울특별시 광진구 긴고랑로9길 28 중곡제일시장 103호,기타 한식,5000,13,4.0,715,이사온 첫날 점심은 짜장면인데 집에서 먹기 싫어서밖으로 나와 망플을 켭니다..1위가...
4,건대우동집,서울특별시 광진구 아차산로 201 동성빌딩 1F,기타 한식,5000,9,4.0,45,혼밥하기 좋은곳.김밥도 괜찮고 잔치국수도 깔끔하니 괜찮다.. 잔치국수와 건대김밥 주...


In [29]:
eat_info['price'].unique()

array(['만원 미만', '만원-2만원', '2만원-3만원', '3만원-4만원', '4만원 이상'], dtype=object)

In [31]:
eat_info['price'].replace('만원 미만',5000, inplace=True)
eat_info['price'].replace('만원-2만원',15000, inplace=True)
eat_info['price'].replace('2만원-3만원',25000, inplace=True)
eat_info['price'].replace('3만원-4만원',35000, inplace=True)
eat_info['price'].replace('4만원 이상',50000, inplace=True)

In [33]:
eat_review.head()


,res_name,user_name,review,rating,p_id,u_id
0,원조할아버지손두부,달려요달려,예전 초당할머니 순두부는 살짝 변해서여길 못따라옴흰순두부는 맛과 향이 살짝 비릴수 ...,5,553,715
1,원조할아버지손두부,ultraus,아차산 등산 후 내려오는 길에 필수코스처럼 되어버린 두부집. 늘 대기가 조금씩 있는...,5,553,453
2,원조할아버지손두부,세상의 모든 음식을 정복하겠어,<다른메뉴 없고 손두부만 파는 집>이 집의 매력포인트 중 하나는 막걸리가 정말 싸다...,5,553,1080
3,원조할아버지손두부,조선의장희빈,등산 후 맛보는 기가막힌 #원조할아버지손두부- 직접 만든 유기농 100% 손두부- ...,5,553,1479
4,원조할아버지손두부,미콩,드디어 먹어본 아차산 순두부!! 같이 가 준 친구들이 이미 전에도 맛본 경험이 있어...,5,553,954


In [30]:
eat_review['rating'].unique()

array(['맛있다', '별로', '괜찮다'], dtype=object)

In [32]:
eat_review['rating'].replace('맛있다',5, inplace=True)
eat_review['rating'].replace('괜찮다',3, inplace=True)
eat_review['rating'].replace('별로',0, inplace=True)

/var/folders/39/19s9x2053p9bhn6svbpwcm_h0000gn/T/ipykernel_43148/3115723822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eat_review['rating'].replace('맛있다',5, inplace=True)
/var/folders/39/19s9x2053p9bhn6svbpwcm_h0000gn/T/ipykernel_43148/3115723822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eat_review['rating'].replace('괜찮다',3, inplace=True)
/var/folders/39/19s9x2053p9bhn6svbpwcm_h0000gn/T/ipykernel_43148/3115723822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [35]:
eat_info.to_csv('../data/translated_eat_info.csv', index=False)
eat_review.to_csv('../data/translated_eat_review.csv', index=False)